In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score, confusion_matrix, mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_train = pd.read_csv('C:/Users/Jay/Desktop/Python/Projects/P1/P1 Data/Consumer_Complaints_train.csv')
df_test = pd.read_csv('C:/Users/Jay/Desktop/Python/Projects/P1/P1 Data/Consumer_Complaints_test_share.csv')

In [3]:
#checking both the datasets
df_train.info()

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 478421 non-null  object
 1   Product                       478421 non-null  object
 2   Sub-product                   339948 non-null  object
 3   Issue                         478421 non-null  object
 4   Sub-issue                     185796 non-null  object
 5   Consumer complaint narrative  75094 non-null   object
 6   Company public response       90392 non-null   object
 7   Company                       478421 non-null  object
 8   State                         474582 non-null  object
 9   ZIP code                      474573 non-null  object
 10  Tags                          67206 non-null   object
 11  Consumer consent provided?    135487 non-null  object
 12  Submitted via                 478421 non-null  object
 13 

In [4]:
#response variable
df_train['Consumer disputed?'].value_counts()

No     376990
Yes    101431
Name: Consumer disputed?, dtype: int64

In [5]:
#checking unique values in the columns
for col in df_train.select_dtypes(['object']).columns:
    print(col, ':' , df_train[col].nunique())

Date received : 1759
Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company public response : 10
Company : 3276
State : 62
ZIP code : 25962
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Date sent to company : 1706
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [6]:
#choosing a subset of data to extract max possible values
print(df_train['Company'].value_counts()[0:40].sum()/478421)
print(df_test['Company'].value_counts()[0:40].sum()/119606)

0.7510330859222317
0.7529471765630487


In [7]:
#choosing a subset of data to extract max possible values
print(df_train['State'].value_counts()[0:25].sum()/478421)
print(df_test['State'].value_counts()[0:25].sum()/119606)

0.8716360694869163
0.8742956038994699


### Data cleaning notes:

- 'Consumer disputed?' is the response variable, change it to 'Y'

- Update col names [remove '?', '-' and spaces]

1. **One Hot Encoding** => product, tags, consent provided, Submitted via, Company response to consumer, Timely response?
[since they have less unique values, wont lead to data explosion]
2. **Drop** => Company ID, zip [ID is not needed, ZIP has ~3K unique values]
3. **tf idf** => issue, narrative [Using NLP]
4. **subset** => Company (top 40), state(top 25) [Has many unique values, but subset provides significant data which can be chosen and converted using one hot encoding ]
5. **isNaN** => Sub-product, Sub-issue, Company public response [has many nulls, thus creating a single col to check if value is present or not]
6. **Date conversion** [checking the difference between dates to understand the resolution time]

In [8]:
#date conversion
df_train['Date received'] = pd.to_datetime(df_train['Date received'], infer_datetime_format=True)
df_train['Date sent to company'] = pd.to_datetime(df_train['Date sent to company'], infer_datetime_format=True)

df_train['date diff'] =  df_train['Date sent to company'] - df_train['Date received']

#df_train.drop(['Date received','Date sent to company'],1,inplace=True)